In [3]:
!pip install pyarrow

     |████████████████████████████████| 40.8 MB 26.9 MB/s eta 0:00:01


In [1]:
import pandas as pd
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression

from sklearn.metrics import mean_squared_error

# Q1. Downloading the data

Download the data for January and February 2023.

Read the data for January. How many columns are there?

In [2]:
nyc_jan_df = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet')

In [3]:
nyc_jan_df.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,2,2023-01-01 00:32:10,2023-01-01 00:40:36,1.0,0.97,1.0,N,161,141,2,9.3,1.00,0.5,0.00,0.0,1.0,14.30,2.5,0.00
1,2,2023-01-01 00:55:08,2023-01-01 01:01:27,1.0,1.10,1.0,N,43,237,1,7.9,1.00,0.5,4.00,0.0,1.0,16.90,2.5,0.00
2,2,2023-01-01 00:25:04,2023-01-01 00:37:49,1.0,2.51,1.0,N,48,238,1,14.9,1.00,0.5,15.00,0.0,1.0,34.90,2.5,0.00
3,1,2023-01-01 00:03:48,2023-01-01 00:13:25,0.0,1.90,1.0,N,138,7,1,12.1,7.25,0.5,0.00,0.0,1.0,20.85,0.0,1.25
4,2,2023-01-01 00:10:29,2023-01-01 00:21:19,1.0,1.43,1.0,N,107,79,1,11.4,1.00,0.5,3.28,0.0,1.0,19.68,2.5,0.00


In [4]:
nyc_jan_df.shape

(3066766, 19)

In [5]:
nyc_feb_df = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-02.parquet')

In [6]:
nyc_feb_df.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee
0,1,2023-02-01 00:32:53,2023-02-01 00:34:34,2.0,0.30,1.0,N,142,163,2,4.4,3.50,0.5,0.0,0.0,1.0,9.40,2.5,0.00
1,2,2023-02-01 00:35:16,2023-02-01 00:35:30,1.0,0.00,1.0,N,71,71,4,-3.0,-1.00,-0.5,0.0,0.0,-1.0,-5.50,0.0,0.00
2,2,2023-02-01 00:35:16,2023-02-01 00:35:30,1.0,0.00,1.0,N,71,71,4,3.0,1.00,0.5,0.0,0.0,1.0,5.50,0.0,0.00
3,1,2023-02-01 00:29:33,2023-02-01 01:01:38,0.0,18.80,1.0,N,132,26,1,70.9,2.25,0.5,0.0,0.0,1.0,74.65,0.0,1.25
4,2,2023-02-01 00:12:28,2023-02-01 00:25:46,1.0,3.22,1.0,N,161,145,1,17.0,1.00,0.5,3.3,0.0,1.0,25.30,2.5,0.00


In [7]:
nyc_feb_df.shape

(2913955, 19)

# Q2. Computing duration

Now let's compute the duration variable. It should contain the duration of a ride in minutes.

What's the standard deviation of the trips duration in January?

In [8]:
# Calculate duration of trips in minutes
nyc_jan_df['duration'] = nyc_jan_df.tpep_dropoff_datetime - nyc_jan_df.tpep_pickup_datetime
nyc_jan_df.duration = nyc_jan_df.duration.apply(lambda td: td.total_seconds() / 60)


In [9]:
nyc_jan_df['duration'].head()

0     8.433333
1     6.316667
2    12.750000
3     9.616667
4    10.833333
Name: duration, dtype: float64

In [10]:
# Calculate the standard deviation of the duration column
duration_std = nyc_jan_df['duration'].std()
print(duration_std)

42.594351241920904


# Q3. Dropping outliers

Next, we need to check the distribution of the duration variable. There are some outliers. Let's remove them and keep only the records where the duration was between 1 and 60 minutes (inclusive)

In [11]:
# Remove outliers (keep only durations between 1 and 60 minutes inclusive)
filtered_nyc_jan_df = nyc_jan_df[(nyc_jan_df['duration'] >= 1) & (nyc_jan_df['duration'] <= 60)].copy()

# Calculate the fraction of records left
fraction_left = len(filtered_nyc_jan_df) / len(nyc_jan_df) 
print(fraction_left*100)

98.1220282212598


# Q4. One-hot encoding

Let's apply one-hot encoding to the pickup and dropoff location IDs. We'll use only these two features for our model.

Turn the dataframe into a list of dictionaries (remember to re-cast the ids to strings - otherwise it will label encode them)

Fit a dictionary vectorizer

Get a feature matrix from it

What's the dimensionality of this matrix (number of columns)?

In [12]:
categorical = ['PULocationID', 'DOLocationID']

filtered_nyc_jan_df[categorical] = filtered_nyc_jan_df[categorical].astype(str)

In [13]:
# Create list of dicts
train_dicts = filtered_nyc_jan_df[categorical].to_dict(orient='records')

# Fit a dictionary vectorizer
dv = DictVectorizer()

feature_matrix = dv.fit_transform(train_dicts)

# Determine the dimensionality of the feature matrix
num_columns = feature_matrix.shape[1]
print(num_columns)

515


# Q5. Training a model

Now let's use the feature matrix from the previous step to train a model.

Train a plain linear regression model with default parameters

Calculate the RMSE of the model on the training data

What's the RMSE on train?

In [14]:
target = 'duration'
y_train = filtered_nyc_jan_df[target].values

lr = LinearRegression()
lr.fit(feature_matrix, y_train)

y_pred = lr.predict(feature_matrix)

mean_squared_error(y_train, y_pred, squared=False)

7.6492610279057605

# Q6. Evaluating the model

Now let's apply this model to the validation dataset (February 2023).

What's the RMSE on validation?

In [ ]:
#Repeat steps for Feb 2023 data

categorical = ['PULocationID', 'DOLocationID']
# Calculate duration of trips in minutes
nyc_feb_df['duration'] = nyc_feb_df.tpep_dropoff_datetime - nyc_feb_df.tpep_pickup_datetime
nyc_feb_df.duration = nyc_feb_df.duration.apply(lambda td: td.total_seconds() / 60)

# Remove outliers (keep only durations between 1 and 60 minutes inclusive)
filtered_nyc_feb_df = nyc_feb_df[(nyc_feb_df['duration'] >= 1) & (nyc_feb_df['duration'] <= 60)].copy()


filtered_nyc_feb_df[categorical] = filtered_nyc_feb_df[categorical].astype(str)


# Create list of dicts
val_dicts = filtered_nyc_feb_df[categorical].to_dict(orient='records')

# Fit a dictionary vectorizer
dv = DictVectorizer()

feature_matrix_feb = dv.fit_transform(val_dicts)

In [ ]:
target = 'duration'
y_train = filtered_nyc_jan_df[target].values
y_val = filtered_nyc_feb_df[target].values

In [ ]:
lr = LinearRegression()
lr.fit(feature_matrix, y_train)

y_pred = lr.predict(feature_matrix_feb)

mean_squared_error(y_val, y_pred, squared=False)